In [11]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import squareform
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import pickle
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split
from scipy.stats import mannwhitneyu
from scipy.spatial.distance import cdist


In [12]:
## Definitions

## Defines

# Data confirmmation :: Nan counts + Nan Ratios, Unique values
def analyse_dataframe(df):
    # NaN 개수, 비율 및 고유 값 개수 계산
    nan_counts = df.isnull().sum()
    nan_ratios = df.isnull().mean() * 100  # 백분율로 표시
    unique_values_count = df.nunique()

    # 결과 데이터 프레임 생성
    analysis_result = pd.DataFrame({
        'NaN Counts': nan_counts,
        'NaN Ratios (%)': nan_ratios,
        'Unique Values Count': unique_values_count
    })

    return analysis_result

def print_progress(iteration, total, prefix='', suffix='', decimals=1, length=50, fill='█', printEnd="\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [13]:
## Configs
base = 'C:/Users/AIWM_PC/KTDB/dd'
out = f'{base}/0716'
out_0827 = f'{base}/0827'

# out_0830 = f'{base}/0903_valid'
out_0830 = f'{base}/1003'

In [14]:
df = pd.read_csv(os.path.join(out,"국내전체_10_29.csv"))
df.set_index(['VARCODE'], inplace=True)
df

,ptmibdrt,ptmisexx,trmsbd14,trmsbd19,trmsbd24,trmsbd25,trmsbd28,trmsbd29,trmspt11,trmspt12,...,trmstaip_4,trmsdmtp_8,trmsdmtp_9,trmsackn_11,trmsackn_50,trmsackn_99,ptmiarcf_8,trmsvgcm_3,trmsvgce_3,trmstaio_2
VARCODE,,,,,,,,,,,,,,,,,,,,,
10542615,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10545593,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10545706,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10545816,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10545884,14.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6304,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0
4043,14.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,NaN,1.0,NaN,0.0,0.0,0.0
9886,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0


In [15]:
df_scd = pd.read_csv(os.path.join(out, "국내전체_10_29_SCD.csv"))

df_scd.set_index(['VARCODE'], inplace=True)
df_scd['Survival'] = df['Survival']
df_scd

,ptmibdrt,ptmisexx,trmspt11,trmspt12,trmspt13,trmspt14,trmspt17,trmspt21,trmspt22,trmspt24,...,trmstmap_0,trmstmap_1_36,trmstmap_37_55,trmstmap_56_69,trmstmap_N,trmstmap__70,trmsitr1_Y_N,trmsitr2_Y_N,trmsitr3_Y_N,TxT_Y_N
VARCODE,,,,,,,,,,,,,,,,,,,,,
10542615,0.770473,-1.567298,7.140482,4.461794,-0.173166,-0.111213,1.521027,-0.042696,-0.101201,0.411462,...,2.639091,-0.235411,-0.255070,-0.153892,-1.547711,-0.182501,-0.600271,-0.319832,-0.13212,-0.611497
10545593,1.036137,-1.567298,7.140482,4.461794,-0.173166,-0.111213,1.521027,-0.042696,-0.101201,0.411462,...,-0.378918,4.247888,-0.255070,-0.153892,-1.547711,-0.182501,-0.600271,3.126645,-0.13212,1.635331
10545706,0.239146,-1.567298,7.140482,-0.224125,-0.173166,-0.111213,1.521027,-0.042696,-0.101201,0.411462,...,-0.378918,-0.235411,-0.255070,-0.153892,0.646115,-0.182501,-0.600271,-0.319832,-0.13212,-0.611497
10545816,0.239146,-1.567298,7.140482,4.461794,-0.173166,-0.111213,1.521027,-0.042696,-0.101201,0.411462,...,-0.378918,-0.235411,-0.255070,-0.153892,0.646115,-0.182501,-0.600271,-0.319832,-0.13212,-0.611497
10545884,0.504809,0.638041,7.140482,4.461794,-0.173166,-0.111213,1.521027,-0.042696,-0.101201,0.411462,...,-0.378918,-0.235411,3.920486,-0.153892,-1.547711,-0.182501,-0.600271,-0.319832,-0.13212,-0.611497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6304,1.301801,-1.567298,7.140482,4.461794,-0.173166,-0.111213,-0.657451,-0.042696,-0.101201,0.411462,...,-0.378918,-0.235411,-0.255070,-0.153892,0.646115,-0.182501,-0.600271,-0.319832,-0.13212,-0.611497
4043,0.504809,0.638041,7.140482,4.461794,-0.173166,-0.111213,1.521027,-0.042696,-0.101201,0.411462,...,-0.378918,-0.235411,3.920486,-0.153892,-1.547711,-0.182501,-0.600271,-0.319832,-0.13212,-0.611497
9886,0.504809,-1.567298,7.140482,4.461794,-0.173166,-0.111213,-0.657451,-0.042696,-0.101201,0.411462,...,-0.378918,-0.235411,-0.255070,-0.153892,0.646115,-0.182501,-0.600271,-0.319832,-0.13212,-0.611497


In [16]:
# Ratio = 24% of train set
df.to_csv(os.path.join(out_0830, "train_df.csv"))
df_scd.to_csv(os.path.join(out_0830, "train_df_scd.csv"))


In [17]:
df_train= pd.read_csv(os.path.join(out,"원핫인코딩해본거_10_22_SCD.csv"))
df_train.set_index(['VARCODE'], inplace=True)

In [18]:
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd

# 1. 거리 계산 행렬인 Z_objects 불러오기 (훈련 데이터로 생성된 것)
with open(os.path.join(out_0830, "Z_objects.pkl"), "rb") as f:
    Z_objects = pickle.load(f)

# 2. 'df_train'에서 클러스터 라벨 할당 (훈련 데이터 길이와 일치하도록)
n_clusters = 4
cluster_labels = fcluster(Z_objects, t=n_clusters, criterion='maxclust')

# 3. 훈련 데이터에서 클러스터 중심 계산
# 'train_data'는 기존 훈련 데이터셋
train_data = df_train.drop(columns=['Survival'])  # 타겟 변수와 불필요한 열 제외

# 클러스터 중심을 저장할 리스트
cluster_centres = []

# 각 클러스터에 속한 데이터의 평균을 계산하여 중심값으로 사용
for label in np.unique(cluster_labels):
    cluster_data = train_data[cluster_labels == label]
    cluster_center = cluster_data.mean(axis=0)  # 클러스터 중심(평균값)을 계산
    cluster_centres.append(cluster_center)

# 클러스터 중심을 배열로 변환
cluster_centres = np.array(cluster_centres)

# 4. 새로운 데이터셋 (검증용 데이터)에 클러스터 라벨 부여
data_for_clustering = df_scd.drop(columns=['Survival']).fillna(0)  # 검증 데이터에서 target 제외, 결측값 처리

# 검증 데이터와 클러스터 중심 간의 거리를 계산
distances = cdist(data_for_clustering, cluster_centres, metric='euclidean')

# 각 검증 데이터 샘플을 가장 가까운 클러스터에 할당
closest_clusters = np.argmin(distances, axis=1) + 1  # 클러스터 인덱스는 1부터 시작하므로 +1

# 검증 데이터프레임에 클러스터 라벨 추가
df_scd['Cluster_Labels'] = closest_clusters
df_scd['pid'] = df_scd.index
df_scd.set_index('pid', inplace=True)
df_scd['Survival'] = df.get('Survival')  # 필요한 경우 Survival 컬럼 유지

# 결과 출력
print("Cluster labels assigned to validation dataset:")
print(df_scd[['Cluster_Labels']])
print()
print("Clustering completed with Cluster Labels to 'df_scd'.")
print(df_scd)


Cluster labels assigned to validation dataset:
          Cluster_Labels
pid                     
10542615               1
10545593               4
10545706               4
10545816               4
10545884               4
...                  ...
6304                   4
4043                   3
9886                   3
9083                   1
5217                   3

[624 rows x 1 columns]

Clustering completed with Cluster Labels to 'df_scd'.
          ptmibdrt  ptmisexx  trmspt11  trmspt12  trmspt13  trmspt14  \
pid                                                                    
10542615  0.770473 -1.567298  7.140482  4.461794 -0.173166 -0.111213   
10545593  1.036137 -1.567298  7.140482  4.461794 -0.173166 -0.111213   
10545706  0.239146 -1.567298  7.140482 -0.224125 -0.173166 -0.111213   
10545816  0.239146 -1.567298  7.140482  4.461794 -0.173166 -0.111213   
10545884  0.504809  0.638041  7.140482  4.461794 -0.173166 -0.111213   
...            ...       ...       ...       

In [19]:
## FEATURE ANALYSIS BY IAS :: 

df_combined = df.merge(df_scd[['Cluster_Labels']], left_index=True, right_index=True, how='left')

def analyze_clusters(df):
    # Cluster groups
    cluster_groups = df.groupby('Cluster_Labels')
    
    # Summary statistics
    summary_stats = cluster_groups.agg(
        total=('Cluster_Labels', 'size'),
        deaths=('Survival', lambda x: (x == 1).sum()),
        survivors=('Survival', lambda x: (x == 0).sum())
    ).astype(int)
    
    # Calculate mortality in the specified format
    summary_stats['mortality'] = summary_stats.apply(
        lambda row: f"{row['survivors']}, {row['deaths']} / {row['total']} ({(row['deaths'] / row['total'] * 100):.2f}%)",
        axis=1
    )

    # Reorder columns as requested
    ordered_columns = [
        'mortality','trmspt11'
    
    ]

    # Mean and standard deviation for the selected columns
    mean_std = cluster_groups[ordered_columns[1:]].agg(['mean', 'std'])  # 'mortality'를 제외한 나머지 열들

    # Combine mean and std_dev into the required format
    for col in ordered_columns[1:]:
        mean_std[(col, 'mean ± std_dev')] = mean_std[(col, 'mean')].round(2).astype(str) + " ± " + mean_std[(col, 'std')].round(2).astype(str)

    # Drop the separate mean and std columns, keep only 'mean ± std_dev'
    mean_std = mean_std.xs('mean ± std_dev', axis=1, level=1)
    
    # Combine summary statistics and mean_std into a single dataframe
    combined_df = summary_stats[['mortality']].join(mean_std)
        
#     combined_df = combined_df[ordered_columns]
    
    return combined_df

# 클러스터 레이블이 이미 존재한다고 가정하고, analyze_clusters 함수를 실행하여 결과를 출력합니다.
# 예를 들어, 클러스터 레이블을 다음과 같이 추가할 수 있습니다.
# bal_df['Cluster_Labels'] = some_clustering_method(bal_df[columns_to_replace])

# 분석 결과를 확인합니다.
analysis_results = analyze_clusters(df_combined)
analysis_results

,mortality,trmspt11
Cluster_Labels,,
1,"308, 0 / 308 (0.00%)",0.18 ± 0.38
2,"6, 12 / 20 (60.00%)",0.2 ± 0.41
3,"180, 7 / 187 (3.74%)",0.23 ± 0.42
4,"108, 1 / 109 (0.92%)",0.28 ± 0.45


In [20]:
## DEATH RATIO
# 'Cluster_Labels'와 'death_label' 확인

def calculate_mortality_rate(ipt_df, target_col):
    # 'Cluster_Labels'와 target_col 확인
    if 'Cluster_Labels' in ipt_df.columns and target_col in ipt_df.columns:
        results_df = pd.DataFrame()

        # 각 클러스터의 사망률 계산
        for i in ipt_df['Cluster_Labels'].unique():  # unique 클러스터 라벨 사용
            cluster_data = ipt_df[ipt_df['Cluster_Labels'] == i]
            total_count = cluster_data.shape[0]
            death_count = (cluster_data[target_col] == 1).sum()
            survive_count = (cluster_data[target_col] == 0).sum()

            # 사망률 계산
            mortality_rate = (death_count / total_count) * 100

            # 결과 저장
            results_df.loc[f'Cluster {i}', 'Total'] = total_count
            results_df.loc[f'Cluster {i}', 'Deaths'] = survive_count
            results_df.loc[f'Cluster {i}', 'Survivors'] = death_count
            results_df.loc[f'Cluster {i}', 'Survivors Rate'] = f"{mortality_rate:.2f}%"
            results_df.loc[f'Cluster {i}', 'Survivors / Total'] = f"{death_count} / {total_count}"

        # 결과 출력
        results_df = results_df.sort_index()
        print(results_df)
    else:
        print(f"Dataframe does not contain required columns 'Cluster_Labels' and '{target_col}'")

# 사용 예시
calculate_mortality_rate(df_combined, 'Survival')

           Total  Deaths  Survivors Survivors Rate Survivors / Total
Cluster 1  308.0   308.0        0.0          0.00%           0 / 308
Cluster 2   20.0     6.0       12.0         60.00%           12 / 20
Cluster 3  187.0   180.0        7.0          3.74%           7 / 187
Cluster 4  109.0   108.0        1.0          0.92%           1 / 109
